# Assignment 1: Time Series Forecast With Python (Seasonal ARIMA)

**Lecturer**: Vincent Claes<br>
**Authors:** Bryan Honof, Jeffrey Gorissen<br>
**Start Date:** 19/10/2018
    
**Objective:** Visualize and predict the future temperatures via ARIMA

**Description:** In this notebook we train our model

In [1]:
import math
import warnings
import datetime

import pandas            as pd
import itertools         as it
import statsmodels.api   as sm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore") # specify to ignore warning messages

In [2]:
data_csv = pd.read_csv('rolmean_data.csv')
data = pd.DataFrame()

# Convert the creation_date column to datetime64
data['dateTime'] = pd.to_datetime(data_csv['dateTime'])
# Convert the value column to float
data['temperature'] = pd.to_numeric(data_csv['temperature'])

# Set the dateTime column as index
data = data.set_index(['dateTime'])

# Sort the dataFrame just to be sure...
data = data.sort_index()

data = data.dropna()

# Double check the results
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 827 entries, 2018-11-11 03:00:00 to 2018-11-19 17:30:00
Data columns (total 1 columns):
temperature    827 non-null float64
dtypes: float64(1)
memory usage: 12.9 KB


In [3]:
data.tail(5)

,temperature
dateTime,
2018-11-19 16:30:00,17.28
2018-11-19 16:45:00,17.28
2018-11-19 17:00:00,17.29
2018-11-19 17:15:00,17.23
2018-11-19 17:30:00,17.21


In [4]:
# Tune Seasonal ARIMA model
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)

# Generate all different combinations of p, q and q triplets
pdq = list(it.product(p, d, q))
print(pdq)

# Generate all different combinations of seasonal p, q and q triplets
# Seasonality is one week (24*7 = 168 hours)
seasonal_pdq = [(x[0], x[1], x[2], 15) for x in list(it.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 15)
SARIMAX: (0, 0, 1) x (0, 1, 0, 15)
SARIMAX: (0, 1, 0) x (0, 1, 1, 15)
SARIMAX: (0, 1, 0) x (1, 0, 0, 15)


In [5]:
train = data

train.index = pd.DatetimeIndex(train.index.values)

result_list = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()

            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, round(results.aic,2)))
            result_list.extend([param, param_seasonal, round(results.aic,2)])
        except:
            print('error')
            continue
            
print('Done!')

ARIMA(0, 0, 0)x(0, 0, 0, 15) - AIC:7146.73
ARIMA(0, 0, 0)x(0, 0, 1, 15) - AIC:5981.76
ARIMA(0, 0, 0)x(0, 1, 0, 15) - AIC:2511.15
ARIMA(0, 0, 0)x(0, 1, 1, 15) - AIC:2470.73
ARIMA(0, 0, 0)x(1, 0, 0, 15) - AIC:2512.12
ARIMA(0, 0, 0)x(1, 0, 1, 15) - AIC:2502.85
ARIMA(0, 0, 0)x(1, 1, 0, 15) - AIC:2477.61
ARIMA(0, 0, 0)x(1, 1, 1, 15) - AIC:2463.29
ARIMA(0, 0, 1)x(0, 0, 0, 15) - AIC:6009.25
ARIMA(0, 0, 1)x(0, 0, 1, 15) - AIC:4937.63
ARIMA(0, 0, 1)x(0, 1, 0, 15) - AIC:1528.76
ARIMA(0, 0, 1)x(0, 1, 1, 15) - AIC:1513.34
ARIMA(0, 0, 1)x(1, 0, 0, 15) - AIC:1531.1
ARIMA(0, 0, 1)x(1, 0, 1, 15) - AIC:1529.73
ARIMA(0, 0, 1)x(1, 1, 0, 15) - AIC:1516.22
ARIMA(0, 0, 1)x(1, 1, 1, 15) - AIC:1511.93
ARIMA(0, 1, 0)x(0, 0, 0, 15) - AIC:-1238.56
ARIMA(0, 1, 0)x(0, 0, 1, 15) - AIC:-1346.14
ARIMA(0, 1, 0)x(0, 1, 0, 15) - AIC:-477.04
ARIMA(0, 1, 0)x(0, 1, 1, 15) - AIC:-1126.66
ARIMA(0, 1, 0)x(1, 0, 0, 15) - AIC:-1270.55
ARIMA(0, 1, 0)x(1, 0, 1, 15) - AIC:-1400.58
ARIMA(0, 1, 0)x(1, 1, 0, 15) - AIC:-759.76
ARIMA(0

In [6]:
print_result = zip(*[iter(result_list)]*3) 
print_result = sorted(print_result, key=lambda x: x[1])

print('Result summary:\n')
print('((p, d, q), (P, D, Q, S), AIC)')
print('------------------------------')
for item in print_result:
    print(item)

Result summary:

((p, d, q), (P, D, Q, S), AIC)
------------------------------
((0, 0, 0), (0, 0, 0, 15), 7146.73)
((0, 0, 1), (0, 0, 0, 15), 6009.25)
((0, 1, 0), (0, 0, 0, 15), -1238.56)
((0, 1, 1), (0, 0, 0, 15), -1408.0)
((1, 0, 0), (0, 0, 0, 15), -1239.11)
((1, 0, 1), (0, 0, 0, 15), -1409.15)
((1, 1, 0), (0, 0, 0, 15), -1537.76)
((1, 1, 1), (0, 0, 0, 15), -1682.09)
((0, 0, 0), (0, 0, 1, 15), 5981.76)
((0, 0, 1), (0, 0, 1, 15), 4937.63)
((0, 1, 0), (0, 0, 1, 15), -1346.14)
((0, 1, 1), (0, 0, 1, 15), -1626.98)
((1, 0, 0), (0, 0, 1, 15), -1214.67)
((1, 0, 1), (0, 0, 1, 15), -1376.46)
((1, 1, 0), (0, 0, 1, 15), -1910.79)
((1, 1, 1), (0, 0, 1, 15), -2120.19)
((0, 0, 0), (0, 1, 0, 15), 2511.15)
((0, 0, 1), (0, 1, 0, 15), 1528.76)
((0, 1, 0), (0, 1, 0, 15), -477.04)
((0, 1, 1), (0, 1, 0, 15), -587.79)
((1, 0, 0), (0, 1, 0, 15), -481.86)
((1, 0, 1), (0, 1, 0, 15), -595.19)
((1, 1, 0), (0, 1, 0, 15), -660.76)
((1, 1, 1), (0, 1, 0, 15), -770.37)
((0, 0, 0), (0, 1, 1, 15), 2470.73)
((0, 0, 1)